In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv("merged_final_2.csv")

C:\Users\alish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,Unnamed: 0,book_id,date_added,date_updated,is_read,rating,read_at,review_id,review_text_incomplete,started_at,...,publication_year,publisher,ratings_count,series,similar_books,text_reviews_count,title,title_without_series,url,work_id
0,0,24396144,Thu Dec 15 15:37:54 -0800 2016,Thu Dec 15 15:37:55 -0800 2016,False,0,NaN,d210e41fcc7e6dcd6ae896844a38a024,NaN,NaN,...,2015.0,Laurence King Publishing,57,[],[],20,Pierre the Maze Detective: The Search for the ...,Pierre the Maze Detective: The Search for the ...,https://www.goodreads.com/book/show/24396144-p...,43980643
1,1,24396144,Tue Apr 26 09:41:08 -0700 2016,Tue Apr 26 09:41:09 -0700 2016,False,0,NaN,fc16284cbd90e9a061ee64a93b2991c2,NaN,NaN,...,2015.0,Laurence King Publishing,57,[],[],20,Pierre the Maze Detective: The Search for the ...,Pierre the Maze Detective: The Search for the ...,https://www.goodreads.com/book/show/24396144-p...,43980643
2,2,24396144,Wed Nov 30 23:29:41 -0800 2016,Wed Nov 30 23:29:42 -0800 2016,False,0,NaN,e00cbe71f37520d59f71201e2bcf7328,NaN,NaN,...,2015.0,Laurence King Publishing,57,[],[],20,Pierre the Maze Detective: The Search for the ...,Pierre the Maze Detective: The Search for the ...,https://www.goodreads.com/book/show/24396144-p...,43980643
3,3,24396144,Wed Nov 18 07:12:11 -0800 2015,Wed Nov 18 07:12:15 -0800 2015,False,0,NaN,23ef4cc13bb162e8dad5ca75e4a30ea4,NaN,NaN,...,2015.0,Laurence King Publishing,57,[],[],20,Pierre the Maze Detective: The Search for the ...,Pierre the Maze Detective: The Search for the ...,https://www.goodreads.com/book/show/24396144-p...,43980643
4,4,23310161,Tue Nov 17 11:37:35 -0800 2015,Tue Nov 17 11:38:05 -0800 2015,True,4,NaN,f4b4b050f4be00e9283c92a814af2670,Fun sequel to the original.,NaN,...,2015.0,Philomel Books,8924,['751058'],"['22249668', '25745002', '23309640', '23735067...",1231,The Day the Crayons Came Home,The Day the Crayons Came Home,https://www.goodreads.com/book/show/23310161-t...,42864407


In [4]:
data.columns

Index(['Unnamed: 0', 'book_id', 'date_added', 'date_updated', 'is_read',
       'rating', 'read_at', 'review_id', 'review_text_incomplete',
       'started_at', 'user_id', 'asin', 'authors', 'average_rating',
       'country_code', 'description', 'edition_information', 'format',
       'image_url', 'is_ebook', 'isbn', 'isbn13', 'kindle_asin',
       'language_code', 'link', 'num_pages', 'popular_shelves',
       'publication_day', 'publication_month', 'publication_year', 'publisher',
       'ratings_count', 'series', 'similar_books', 'text_reviews_count',
       'title', 'title_without_series', 'url', 'work_id'],
      dtype='object')

In [61]:
ds = data[['user_id','rating', 'book_id', 'title', 'read_at', 'description']]

In [62]:
# Select only those users having more than 20 
num = pd.DataFrame(ds.groupby('book_id')['rating'].count())
num = num[num['rating']>=10]
num.reset_index()

df2 = pd.merge(num,ds,on='book_id').drop_duplicates()
df2.head()

,book_id,rating_x,user_id,rating_y,title,read_at,description
0,103,14,8842281e1d1347389f2ab93d60773d4d,4,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
1,103,14,9a88311cb8f79be7a0a84dba2157534e,0,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
2,103,14,653a89a7bb5a9fe1716e01d879d423c9,3,"God Emperor of Dune (Dune Chronicles, #4)",Fri Aug 17 14:17:30 -0700 2012,"Centuries have passed on Dune, and the planet ..."
3,103,14,5a3f081f27a0004518b6544074545018,4,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
4,103,14,14ea78d43c0a440cd2de9f3fdaa3daf6,0,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."


In [63]:
ds = df2.rename(columns={"rating_y": "rating"})
ds.head()

,book_id,rating_x,user_id,rating,title,read_at,description
0,103,14,8842281e1d1347389f2ab93d60773d4d,4,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
1,103,14,9a88311cb8f79be7a0a84dba2157534e,0,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
2,103,14,653a89a7bb5a9fe1716e01d879d423c9,3,"God Emperor of Dune (Dune Chronicles, #4)",Fri Aug 17 14:17:30 -0700 2012,"Centuries have passed on Dune, and the planet ..."
3,103,14,5a3f081f27a0004518b6544074545018,4,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."
4,103,14,14ea78d43c0a440cd2de9f3fdaa3daf6,0,"God Emperor of Dune (Dune Chronicles, #4)",NaN,"Centuries have passed on Dune, and the planet ..."


In [64]:
ds = ds.dropna(subset = ['read_at','description']).reset_index()

In [65]:
ds = ds[['book_id', 'title', 'description']].drop_duplicates().reset_index()

In [67]:
ds.to_csv('dedup_bookid.csv')

In [68]:
ds.shape

(11966, 4)

In [69]:
ds.head()

,index,book_id,title,description
0,0,103,"God Emperor of Dune (Dune Chronicles, #4)","Centuries have passed on Dune, and the planet ..."
1,1,106,Dune Messiah (Dune Chronicles #2),Dune Messiahcontinues the story of the man Mua...
2,25,234,The Complete Verse and Other Nonsense,This collection demonstrates the varied ways i...
3,26,236,The Complete Nonsense of Edward Lear,"The Owl and the Pussy-Cat, Calico Pie and The ..."
4,30,355,Job: A Comedy of Justice,"After he firewalked in Polynesia, the world wa..."


In [70]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['description'])

In [73]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_similarities = linear_kernel(W, W)

In [75]:
results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['book_id'][i]) for i in similar_indices]

    results[row['book_id']] = similar_items[1:]
    
print('done!')
#print (results)

done!


In [76]:
def item(id):
    #return ds.loc[ds['user_id'] == id]['description'].tolist()[0].split(' - ')[0]
    return ds.loc[ds['book_id'] == id]['title'].tolist()[0].split(' - ')[0]

In [78]:
# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    reccos = pd.DataFrame(columns=['title'])
    for rec in recs:
        reccos = reccos.append({'title':item(rec[1])},ignore_index=True)
        reccos = reccos.drop_duplicates()
#        reccos = reccos[1:]
    return reccos



In [79]:
recommend(item_id=23310161, num=5)
#item('d1e368a7d2870eb6fbf6e0d350568a2d')

Recommending products similar to The Day the Crayons Came Home...
-------


,title
0,Once Upon an Alphabet: Short Stories for All t...
1,Stuck
2,Rocket Writes a Story
3,"Lost and Found (The Boy, #2)"
4,Fog Island


In [80]:
recommend(item_id=29241319, num=5)

Recommending products similar to Harry Potter and the Chamber of Secrets (Harry Potter, #2)...
-------


,title
0,Harry Potter and the Sorcerer's Stone (Harry P...
1,Harry Potter and the Philosopher's Stone
2,Harry Potter and the Deathly Hallows (Harry Po...
3,Here at Hogwarts
4,Harry Potter and the Philosopher's Stone (Harr...


In [82]:
recommend(item_id=8663600, num=5)

Recommending products similar to Mockingjay (The Hunger Games, #3)...
-------


,title
0,"Mockingjay (The Hunger Games, #3)"
1,"Executive Orders (Jack Ryan Universe, #9)"
2,"Captives (Safe Lands, #1)"
3,A Child's Calendar
4,"Deep Down (Jack Reacher, #16.5)"


In [94]:
recommend(item_id=10514736, num=10)[1:6]

Recommending products similar to Wuthering Heights...
-------


,title
1,The Secret Garden
2,"Your Next Breath (Catherine Ling, #4)"
3,The Confessions of Catherine de Medici
4,Becoming Jane Eyre
5,Knight of Desire (All the King's Men #1)
